In [ ]:
import pandas as pd
import ollama
import re
from tqdm import tqdm

class AnswerEvaluator:
    def __init__(self, model_name="qwen2.5"):
        self.model = model_name
        self.system_prompt = """
        İki cevap arasındaki semantik benzerliği 0-20 arası puanla.
        Açıklama, yorum veya ek metin EKLEMEYİN.
        Örnek geçerli çıktı: 16
        Örnek geçersiz çıktı: 16 puan
        """
        
    def load_data(self, excel_path: str):
        """Excel verisini yükler ve temizler"""
        try:
            df = pd.read_excel(excel_path)
            df = df[['Student ID', 'student_answers', 'Hoca Cevabı']]
            df['student_answers'] = df['student_answers'].apply(self.clean_text)
            df['Hoca Cevabı'] = df['Hoca Cevabı'].apply(self.clean_text)
            return df.dropna()
        except Exception as e:
            print(f"Hata: {str(e)}")
            return pd.DataFrame()

    def clean_text(self, text: str) -> str:
        """Metni normalleştirme"""
        text = re.sub(r'\s+', ' ', str(text)).lower().strip()
        return re.sub(r'[^\w\s.,!?]', '', text)

    def calculate_similarity(self, student_answer: str, correct_answer: str) -> float:
        """İki cevap arasındaki benzerlik puanını hesaplar"""
        try:
            response = ollama.generate(
                model=self.model,
                prompt=f"Öğrenci Cevabı: {student_answer}\nDoğru Cevap: {correct_answer}",
                system=self.system_prompt,
                options={"temperature": 0.0, "max_tokens": 50}
            )
            return float(response['response'].strip())
        except Exception as e:
            print(f"Hata: {str(e)}")
            return 0.0

    def evaluate_answers(self, excel_path: str, output_file: str = "degerlendirme_raporu.xlsx"):
        """Ana değerlendirme fonksiyonu"""
        df = self.load_data(excel_path)
        if df.empty:
            return

        results = []
        for _, row in tqdm(df.iterrows(), total=len(df)):
            similarity = self.calculate_similarity(
                row['student_answers'],
                row['Hoca Cevabı']
            )
            
            results.append({
                'Student ID': row['Student ID'],
                'Öğrenci Cevabı': row['student_answers'],
                'Benzerlik Puanı': similarity,
                'Doğru Cevap': row['Hoca Cevabı']
            })
        
        report_df = pd.DataFrame(results)
        report_df.to_excel(output_file, index=False)
        print(f"\nRapor oluşturuldu: {output_file}")
        return report_df

if __name__ == "__main__":
    evaluator = AnswerEvaluator(model_name="qwen2.5")
    
    # Excel dosyasını belirtin
    excel_path = "Son_Excel.xlsx"
    output_file = "Öğrenci_Degerlendirme_Raporu.xlsx"
    
    # Değerlendirmeyi başlat
    print("Değerlendirme başlıyor...")
    report = evaluator.evaluate_answers(excel_path, output_file)
    
    if report is not None:
        print("\nİlk 5 kayıt önizleme:")
        print(report.head())

Değerlendirme başlıyor...


  3%|▎         | 3/88 [00:13<05:38,  3.98s/it]

Hata: could not convert string to float: '16\n\nİki cevap arasındaki benzerlik şu noktaları içeriyor:\n\n1. Hem cevap hem de doğru cevapta kimyasal bileşimi aynı olan maddelerin varlığı vurgulanıyor.\n2. Farkların moleküler yapının farklı olmasıyla açıklanması öneriliyor.\n\nFarklılıklar ise şu şekilde:\n\n- Doğru cevap daha geniş bir kapsamda (isomerizm) ve çeşitli faktörler (atom düzeni, uzay konforu, bağlamalar) üzerinde duruyor.\n- Öğrenci cevabı daha spesifik olarak allotropleri (diamant ve grafit) örneklendirdi.\n\nBu nedenle, iki yanıtın temel fikirlerini paylaştığı için 16 puan verilmiştir.'


  5%|▍         | 4/88 [00:16<04:48,  3.43s/it]

Hata: could not convert string to float: '12\n\nÖğrenci cevabı, doğru cevaba tam olarak uymuyor ve bazı kavramsal hatalar var. Öğrencinin yazdığı "this name is izonom or isonome" ifadesi doğru bir terim tanımlaması değil ve "isomerism" kelimesini de kullanmıyor. Ancak, genel fikir olan farklı moleküler yapıların fiziksel özellikler üzerinde etkisi olmaya çalıştığı için puanı 12 olarak belirledim.'


...

Hata: could not convert string to float: '12\n\nÖğrenci cevabı, doğru cevaba tamamen yakındır ve temel fikirleri doğru bir şekilde ifade eder. Ancak, daha detaylı ve uzun formüle edilmiş doğru cevabın suntuunu ve bazı kilit terimleri (isomerism, atom arrangement, spatial configuration, bonding patterns) içermediği için puanı azaltım gereklidir.'


 98%|█████████▊| 86/88 [02:54<00:04,  2.01s/it]

Hata: could not convert string to float: '14\n\nBu cevaplar arasındaki benzerlik şu noktaları içeriyor:\n\n1. Her iki yanıt da moleküllerin yapısı farklı olabileceğini belirtiyor.\n2. Farklı fiziksel özelliklere yol açan yapısal farklılıkların varlığını vurguluyor.\n\nFarklı noktalar ise:\n\n- Doğru cevap daha spesifik ve detaylı bilgileri içeriyor (izomerizm, atom düzeni, uzayelastik yapı, bağlamalar gibi).\n- Öğrenci cevabı daha kısa ve genel bir açıklama sunuyor.'


100%|██████████| 88/88 [02:57<00:00,  2.01s/it]

Hata: could not convert string to float: '12\n\nBu cevaplar arasında benzerlikler var, ancak doğru cevap daha spesifik ve teknik terimleri kullanarak allotropi konusunu daha iyi açıklamaktadır. Öğrenci cevabı ise allotrope hakkında genel bir fikir veriyor, ancak doğru cevap isomerizmi ve moleküler yapının nasıl etkilendiğini daha ayrıntılı açıklıyor.'

Rapor oluşturuldu: Öğrenci_Degerlendirme_Raporu.xlsx

İlk 5 kayıt önizleme:
    Student ID                                     Öğrenci Cevabı  \
0  20160807006  nuclears theory 3d structure cube added or rem...   
1  20160807009  because of count of elements. mostly about car...   
2  20170808017  allotrops such as diamond and graphit they hav...   
3  20170808045  this name is izonom or isonome this caused fro...   
4  20190808008  because of elements energy and pyhsical featur...   

   Benzerlik Puanı                                        Doğru Cevap  
0              4.0  compounds with the same chemical composition b...  
1         